# Feature preparation
Right here the features for the ml models are created and filtered. Then a csv with only the important features is saved.

In [49]:
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [50]:
path_trips_hexagons=os.path.join(os.getcwd(), "..", "data", "output", "Trips_Hexagons.csv")
path_hexagons=os.path.join(os.getcwd(), "..", "data", "output", "hexagons-dresden.csv")
path_trips = os.path.join(os.getcwd(), "..", "data", "output", "Trips.csv")
print(path_trips_hexagons)

C:\Users\lenovo\Documents\GitHub\AAA_Jupyter_Explorers\notebooks\..\data\output\Trips_Hexagons.csv


In [51]:
df = pd.read_csv(path_trips)
df.head()

,Unnamed: 0,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,...,p_terminal_type_end,p_bike_types_end,MESS_DATUM,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms,trip_duration
0,0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,...,NaN,"{""15"": 1}",2019-01-20 00:00:00,-3.3,75.3,0.0,0.0,0.0,4.3,28.0
1,1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,...,NaN,"{""15"": 1}",2019-01-20 00:00:00,-3.3,75.3,0.0,0.0,0.0,4.3,10.0
2,2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,...,free,"{""undefined"": 1, ""15"": 2}",2019-01-20 00:00:00,-3.3,75.3,0.0,0.0,0.0,4.3,26.0
3,3,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,...,NaN,"{""undefined"": 3, ""15"": 2}",2019-01-20 00:00:00,-3.3,75.3,0.0,0.0,0.0,4.3,28.0
4,4,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,...,NaN,"{""undefined"": 1}",2019-01-20 00:00:00,-3.3,75.3,0.0,0.0,0.0,4.3,2.0


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550350 entries, 0 to 550349
Data columns (total 47 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               550350 non-null  int64  
 1   p_bike_racks_start       550350 non-null  int64  
 2   p_spot_start             550350 non-null  bool   
 3   p_booked_bikes_start     550350 non-null  int64  
 4   p_place_type_start       550350 non-null  int64  
 5   datetime_start           550350 non-null  object 
 6   b_number_start           550350 non-null  int64  
 7   p_uid_start              550350 non-null  int64  
 8   p_bikes_start            550350 non-null  int64  
 9   p_lat_start              550350 non-null  float64
 10  b_electric_lock_start    550350 non-null  bool   
 11  b_bike_type_start        550350 non-null  int64  
 12  p_name_start             550350 non-null  object 
 13  p_free_racks_start       550350 non-null  int64  
 14  p_ad

In [53]:
features.isnull().sum()

p_bike_racks_start       0
p_booked_bikes_start     0
p_place_type_start       0
b_number_start           0
p_uid_start              0
p_bikes_start            0
p_lat_start              0
b_bike_type_start        0
p_free_racks_start       0
p_lng_start              0
b_boardcomputer_start    0
air_deg                  0
air_hum                  0
rain_mm                  0
rain_yn                  0
sun_hour                 0
wind_ms                  0
trip_duration            0
p_spot_start             0
b_electric_lock_start    0
p_maintenance_start      0
dtype: int64

## Drop not needed columns
Drop end information and not usefull columns.
The features are cleaned in three steps. <br>
- features_1 => drop not usefull columns 
- features_2 => drop end information
- features_3 => drop object types

In [54]:
print("Drop not int columns and not usefull")
# Unnamed: 0 => index column
# p_terminal_type_end => NaN values
features_1 = df.drop(["Unnamed: 0", "p_terminal_type_start", "p_number_start", "b_pedelec_battery_start"], axis=1)
print("Drop end information")
features_2 = features_1.drop(
    ["p_bike_racks_end",
     "p_spot_end", 
     "p_booked_bikes_end", 
     "p_place_type_end", 
     "datetime_end",
     "p_uid_end",
     "p_bikes_end",     
     "p_lat_end",
     "p_name_end",
     "p_free_racks_end",
     "p_address_end",
     "p_number_end",
     "p_lng_end",
     "p_maintenance_end",
     "p_terminal_type_end", 
     "p_bike_types_end"], 
    axis=1
)
print("Drop object types...")
features_3 = features_2.drop(["datetime_start", "p_name_start", "p_address_start", "b_lock_types_start", "p_bike_types_start", "MESS_DATUM"], axis=1)

Drop not int columns and not usefull
Drop end information
Drop object types...


### Dummy
Create Dummy variables for all booleans

In [55]:
p_spot_start = features_3["p_spot_start"].astype(int)
b_electric_lock_start = features_3["b_electric_lock_start"].astype(int)
p_maintenance_start = features_3["p_maintenance_start"].astype(int)

# weekend = p_df["Weekend"].astype(int)
features = features_3.drop(["p_spot_start", "b_electric_lock_start", "p_maintenance_start"], axis=1)
features = pd.concat([features, p_spot_start, b_electric_lock_start, p_maintenance_start], axis=1)

### Feature Engineering

In [56]:
# features["XYZ"] = np.square(features["XXX"])

In [57]:
# Save features
features.to_csv(os.path.join(os.getcwd(), "..", "data", "output", "Features.csv"))

### Split Data

In [58]:
features_X = features.drop("trip_duration", axis=1)
features_y = features["trip_duration"]
X_train, X_test, y_train, y_test = train_test_split(features_X, features_y, random_state=42, test_size=0.3)

### Scaling and PCA

In [59]:
# STANDARD SCALER
st_scaler = StandardScaler()
# fit scaler only on training set not on test set
st_scaler.fit(X_train)

# Save Scaler Object
obj = st_scaler
filename = "Standard_Scaler.pkl"
pickle.dump(obj, open(os.path.join(os.getcwd(), "..", "data", "output", "models", filename), "wb"))

X_train_scaled = st_scaler.transform(X_train)

In [60]:
# PCA
pca = PCA(n_components=13) # Number of components is set here
pca.fit(X_train_scaled)
pca_explained_variance = pca.explained_variance_ratio_
print("Var explained:", pca_explained_variance)
print("Sum var explained", sum(pca_explained_variance))

# Save PCA Object
obj = pca
filename = "PCA.pkl"
pickle.dump(obj, open(os.path.join(os.getcwd(), "..", "data", "output", "models", filename), "wb"))

Var explained: [0.16186261 0.12393358 0.09965512 0.09761776 0.08601056 0.06386806
 0.05762055 0.05310806 0.05098027 0.04883361 0.04255657 0.03075712
 0.02592038]
Sum var explained 0.9427242635705549
